# Recommendations Using Hybrid solutions <sup>1</sup>

Lots of examples of systems that try to combine the strengths of the two main approaches.
This can be done in a number of ways:

- Combine the predictions of a content-based system and a collaborative system.
- Incorporate content-based techniques into a collaborative approach.
- Incorporate collaborative techniques into a content-based approach.
- Unifying model.

Possible

## Imports

In [1]:
import numpy as np
import pandas as pd

## Load Data

Only loading a subset of the original data set for proof of concept reasons.

In [2]:
# 80/20 split earlier
df_train = pd.read_csv('../Data/training_data_subset.csv')
df_test = pd.read_csv('../Data/testing_data_subset.csv')

In [3]:
# Also, load the entire original dataset to make a user index.
# Note: If using the full train/test dataset several of the methods below can takes 2-6 hours to run.
df_original = pd.read_csv('../Data/eda_data.csv')

In [4]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1083170 entries, 0 to 1083169
Data columns (total 14 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   category    1083170 non-null  object 
 1   title       1083170 non-null  object 
 2   also_buy    926546 non-null   object 
 3   brand       1075197 non-null  object 
 4   rank        1039163 non-null  object 
 5   also_view   577060 non-null   object 
 6   main_cat    1081896 non-null  object 
 7   price       750231 non-null   float64
 8   asin        1083170 non-null  object 
 9   overall     1083170 non-null  float64
 10  verified    1083170 non-null  bool   
 11  reviewerID  1083170 non-null  object 
 12  vote        149247 non-null   float64
 13  style       559212 non-null   object 
dtypes: bool(1), float64(3), object(10)
memory usage: 108.5+ MB


In [5]:
import logging
logging.basicConfig(filename='../Logging/troubleshoot_index.log', filemode='w+', level=logging.DEBUG)

### Create a user index
To retrieve information given a specific user_id in a more convenient way.

In [6]:
# The key features and ids from earlier analysis.
# Note that additional features could be included if desired; except for the target feature: 'overall'.
user_info = df_original[['title', 'also_buy', 'also_view', 'price', 'rank', 'asin', 'reviewerID', 'vote']]
user_info.set_index('reviewerID', inplace=True)
user_info.head(2)

,title,also_buy,also_view,price,rank,asin,vote
reviewerID,,,,,,,
A1J205ZK25TZ6W,Lipton Yellow Label Tea (loose tea) - 450g,"['B00886E4K0', 'B00CREXSHY', 'B001QTRGAQ', 'B0...","['B00CREXSHY', 'B001QTRGAQ', 'B000JSQK70', 'B0...",12.46,"30,937 in Grocery & Gourmet Food (",4639725043,8.0
ACOICLIJQYECU,Lipton Yellow Label Tea (loose tea) - 450g,"['B00886E4K0', 'B00CREXSHY', 'B001QTRGAQ', 'B0...","['B00CREXSHY', 'B001QTRGAQ', 'B000JSQK70', 'B0...",12.46,"30,937 in Grocery & Gourmet Food (",4639725043,9.0


In [7]:
def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

In [8]:
def evaluate(estimate_f):
    """ RMSE-based predictive performance evaluation with pandas. """
    
    ids_to_estimate = zip(df_test.reviewerID, df_test.asin)
    estimated = np.array([estimate_f(u,i) for (u,i) in ids_to_estimate])
    real = df_test.overall.values
    return compute_rmse(estimated, real)

In [9]:
class CollaborativeRecommendation:
    """ Collaborative filtering using an implicit sim(u,u'). """

    def __init__(self, feature):
        """ Prepare data structures for estimation. """
        self._feature = feature
        self.means_by_feature = df_train.pivot_table('overall', index='asin', columns=self.feature)

    @property
    def feature(self):
        return self._feature
        
    def estimate(self, reviewer_id, product_id):
        """ Mean ratings by other users of the same feature. """
        
        if product_id not in self.means_by_feature.index: 
            return 4.0
        
        user_feature = user_info.loc[reviewer_id, self.feature]
        if ~np.isnan(self.means_by_feature.loc[product_id, user_feature]):
            return self.means_by_feature.loc[product_id, user_feature]
        else:
            return self.means_by_feature.loc[product_id].mean()
        

In [10]:
gender = CollaborativeRecommendation('also_view')
print('RMSE for Gender: %s' % evaluate(gender.estimate))

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index([                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         nan,\n                                                                                                                                                   